In [1]:
import requests
import selenium
from selenium import webdriver
import pyquery
#import tesserocr  
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import bs4
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium import webdriver
import time
import wget
import random
import paramiko
import os
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/LuZhang/Downloads/chromedriver_win32/chromedriver_win32 (2)/chromedriver.exe')


C:\Users\LuZhang\AppData\Local\Temp\ipykernel_4496\1884599241.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='C:/Users/LuZhang/Downloads/chromedriver_win32/chromedriver_win32 (2)/chromedriver.exe')


In [3]:
driver.get("https://www.douyin.com/")

In [14]:
#Get current page's html and download link
def GetPage():
    #driver.get(driver.current_url)
    global CurrentURL
    CurrentURL = driver.current_url
    html = driver.page_source
    global SoupedHtml
    SoupedHtml=BeautifulSoup(html,features='html.parser')
    PatternVideo = re.compile('<source class="" src="//www..*?type="',re.S)
    DF_PatternVideo_With_Redundancy = pd.DataFrame(re.findall(PatternVideo, str(SoupedHtml)))
    #PatternVideo = re.compile('<source class="" src="//.*?type="',re.S)
    PatternVideo = re.compile('<source class="" src="//www..*?type="',re.S)
    DF_PatternVideo_With_Redundancy = pd.DataFrame(re.findall(PatternVideo, str(SoupedHtml)))
    DF_PatternVideo_With_Redundancy
    DF_PatternVideo = pd.DataFrame(DF_PatternVideo_With_Redundancy[0].apply(lambda x:x[24:-8]).tolist())
    global DownloadLink
    DownloadLink = "http://" + str(DF_PatternVideo.loc[0,0])


In [5]:
# Next Page Button Click
def NextPage():
    actions = ActionChains(driver)
    #actions.click().perform()
    ActionChains(driver).key_down(Keys.DOWN).send_keys("s").perform()
    

def PreviousPage():
    actions = ActionChains(driver)
    #actions.click().perform()
    ActionChains(driver).key_down(Keys.DOWN).send_keys("w").perform()
    

In [6]:
def GetTime():
    global CurrentTime
    CurrentTime = time.strftime("%Y%m%d_%H%M%S", time.localtime()).replace(' ','')
    return CurrentTime

In [7]:
#Download the video
def Download():
    #CurrentTime = time.strftime("%Y%m%d_%H:%M:%S", time.localtime()).replace(' ','')
    global DownloadPath
    DownloadPath='C:/Lu/Code/FormalCode/Video'+ str(CurrentTime) + '.mp4'
    wget.download(DownloadLink,DownloadPath)
    

In [8]:
def get_metadata_by_regex(regex,a,b,c,d):
    Pattern = re.compile(regex,re.S)
    DF_Pattern_with_Redundancy = pd.DataFrame(re.findall(Pattern, str(SoupedHtml)))
    DF_Pattern = DF_Pattern_with_Redundancy.loc[a,b][c:d]
    return DF_Pattern


In [16]:
# userName=get_metadata_by_regex('<span>@.*?</span>',1,0,7,-7)
# heartNumber = get_metadata_by_regex('<div class="PK0tKym9 TwlaIXPW">.*?</div>',0,0,31,-6)
# commentNumber = get_metadata_by_regex('<div class="hfgGrUTS">.*?</div>',0,0,22,-6)
# collectNumber=get_metadata_by_regex('<div class="qc4FQDvn">.*?</div>',0,0,22,-6)
#content = driver.find_element(By.XPATH,'//*[@id="video-info-wrap"]/div[1]/div[2]/div/div/span/span/span[1]/span/span/span').get_attribute('innerHTML')

In [222]:
#make csv
df = pd.DataFrame(columns=['Time','user_Name','heart_Number','comment_Number','collect_Number','content'])
df.loc[0] = [CurrentTime,userName,heartNumber,commentNumber,collectNumber,content]
df.to_csv('C:/Lu/Code/FormalCode/NewCSV/MetaDATA.csv',index=False,sep=',')

In [ ]:
#Start the crawler
while True:
    try:
        GetPage()
        GetTime()
        Download()

        userName=get_metadata_by_regex('<span>@.*?</span>',1,0,7,-7)
        heartNumber = get_metadata_by_regex('<div class="PK0tKym9 TwlaIXPW">.*?</div>',0,0,31,-6)
        commentNumber = get_metadata_by_regex('<div class="hfgGrUTS">.*?</div>',0,0,22,-6)
        collectNumber=get_metadata_by_regex('<div class="qc4FQDvn">.*?</div>',0,0,22,-6)
        content = driver.find_element(By.XPATH,'//*[@id="video-info-wrap"]/div[1]/div[2]/div/div/span/span/span[1]/span/span/span').get_attribute('innerHTML')
        
        data_csv=pd.read_csv(r'C:\Lu\Code\FormalCode\NewCSV\MetaDATA.csv')
        data_csv.loc[len(data_csv)] = [CurrentTime,userName,heartNumber,commentNumber,collectNumber,content]
        data_csv.to_csv('C:/Lu/Code/FormalCode/NewCSV/MetaDATA.csv',index=False,sep=',')

        

        transport = paramiko.Transport(('@@@@', @@@@))
        transport.connect(username='@@@@', password='@@@@')
        sftp = paramiko.SFTPClient.from_transport(transport)#如果连接需要密钥，则要加上一个参数，hostkey="密钥"
        sftp.put(DownloadPath, r"@@@@/Info.csv{}{}".format(CurrentTime,".mp4"))#将本地的Windows.txt文件上传至服务器/root/Windows.txt
        transport.close()

        os.remove(DownloadPath)

        time.sleep(int(random.randint(2,55)))

        NextPage()
    
    except:
        print((time.strftime("%Y%m%d_%H"+"."+"%M"+"."+"%S", time.localtime()).replace(' ',''))+ "error")
        driver.get(driver.current_url)
        pass
        continue
    